##### Project: Opioid Exposed Infant Covariates
##### Investigator: Stephen Patrick, Sarah Loch
##### Programmers: Sander Su, Chris Guardo
##### Date Created: 01/17/23
##### Last Modified: 09/30/25

#### Notes:
This notebook depends on the gestational age notebook in the pipeline and pulls all HIV and HCV related LOINC codes from a patient’s medical records around the pregnancy period 


In [0]:
%run "../Project_modules"

#### HIV-positive Variable: HCV-positive
##### search the information about 2 parts:
- Lab list: Sheet 10 - mat HCV and HIV

In [0]:
phenotype_table_location = " ***Insert file location*** "
phenotype_table=spark.sql(f"SELECT * FROM {phenotype_table_location}")
sheet_name="phenotyping.mprint_hcv_hiv_lab"

### combine short names and long names into a search term list 
search_term=get_search_term(sheet_name)
search_term.createOrReplaceTempView("search_term")

##### hcv or hiv positive/present/reactive from lab report

In [0]:
sql=f"""
      select * from
      
      (select * from {meas_table} a
      inner join (select * from search_term) b on lower(trim(measurement_source_value)) LIKE '%' || b.search_term || '%'
      
      where lower(value_source_value) like '%positive%' or lower(value_source_value) like '%present%' or 
      lower(value_source_value) like '%reactive%')  c
       
    """
hcv_hiv_lab_positive = spark.sql(sql)
hcv_hiv_lab_positive.createOrReplaceTempView("hcv_hiv_lab_positive")

##### hcv or hiv quantitative value from lab report

In [0]:
mom_meas_df=spark.sql(f"select * from {meas_table} where person_id in (select distinct mom_person_id from global_temp.ega_w33_or_uncertain_gestation_date)")
mom_meas_df.createOrReplaceTempView("mom_meas_df")

sql=f"""
       select * from
       (select * from mom_meas_df a
       inner join (select * from search_term) b on lower(trim(measurement_source_value)) LIKE '%' || b.search_term || '%'
       where value_as_number is not null)  c
    
    """
hcv_hiv_lab_quantative= spark.sql(sql)
hcv_hiv_lab_quantative.createOrReplaceTempView("hcv_hiv_lab_quantative")

##### Report within time window: From start_gestation_date to 'birth date + 30 days'

In [0]:
def report_within_time_window(report_name):
    
    sql=f"""
           select * from {report_name} a
           inner join  (select mom_person_id, baby_person_id,baby_dob,start_gestation_date,
           length_of_gestation from global_temp.ega_w33_or_uncertain_gestation_date) b
           on a.person_id = b.mom_person_id
           where measurement_date >= start_gestation_date and measurement_date < date_add(baby_dob, 30);
       """
    df = spark.sql(sql)
    return df

##### for lab positive report

In [0]:
hcv_hiv_lab_positive_prenatal = report_within_time_window("hcv_hiv_lab_positive")
hcv_hiv_lab_positive_prenatal.name='hcv_hiv_lab_positive_prenatal'
register_parquet_global_view(hcv_hiv_lab_positive_prenatal)

##### for HCV/HIV lab quantative value report

In [0]:
hcv_hiv_lab_quantative_prenatal= report_within_time_window("hcv_hiv_lab_quantative")
hcv_hiv_lab_quantative_prenatal.name='hcv_hiv_lab_quantative_prenatal'
register_parquet_global_view(hcv_hiv_lab_quantative_prenatal)

##### Prenatal Opioid Exposure

In [0]:
def prenatal_opioid_exposure(prenatal_df_name,lab_name):
    
   sql=f"""
       select * from global_temp.{prenatal_df_name}
       where (mom_person_id,baby_person_id) in 
       (
        select mom_person_id,baby_person_id
        from {phenotype_table}
        where gestational_age_w33_or_uncertain = 1 and live_birth_code=1 and critical_illness_4cpt = 0 
        and respiratory_procedure_code = 0 and   fetal_anomalies_code =0 
        and (mom_oud_inpatient = 1 or mom_oud_outpatient=1 or mom_drug = 1 or mom_drug_in_note = 1)
       )
       and lab='{lab_name}'; 
      """
   df = spark.sql(sql)
   return df


##### For lab positive report

In [0]:
hcv_hiv_lab_positive_prenatal_opioid_hcv = prenatal_opioid_exposure("hcv_hiv_lab_positive_prenatal","HCV")
hcv_hiv_lab_positive_prenatal_opioid_hiv = prenatal_opioid_exposure("hcv_hiv_lab_positive_prenatal","HIV")

hcv_hiv_lab_positive_prenatal_opioid_hcv.name='hcv_hiv_lab_positive_prenatal_opioid_hcv'
register_parquet_global_view(hcv_hiv_lab_positive_prenatal_opioid_hcv)

hcv_hiv_lab_positive_prenatal_opioid_hiv.name='hcv_hiv_lab_positive_prenatal_opioid_hiv'
register_parquet_global_view(hcv_hiv_lab_positive_prenatal_opioid_hiv)

df_inspection("global_temp.hcv_hiv_lab_positive_prenatal_opioid_hcv","all")

##### For quantative report

In [0]:
hcv_hiv_lab_quantative_prenatal_opioid_hcv=prenatal_opioid_exposure("hcv_hiv_lab_quantative_prenatal","HCV")
hcv_hiv_lab_quantative_prenatal_opioid_hcv.name='hcv_hiv_lab_quantative_prenatal_opioid_hcv'
register_parquet_global_view(hcv_hiv_lab_quantative_prenatal_opioid_hcv)

df_inspection("global_temp.hcv_hiv_lab_quantative_prenatal_opioid_hcv","all")

hcv_hiv_lab_quantative_prenatal_opioid_hiv=prenatal_opioid_exposure("hcv_hiv_lab_quantative_prenatal","HIV")
hcv_hiv_lab_quantative_prenatal_opioid_hiv.name='hcv_hiv_lab_quantative_prenatal_opioid_hiv'
register_parquet_global_view(hcv_hiv_lab_quantative_prenatal_opioid_hiv)

df_inspection("global_temp.hcv_hiv_lab_quantative_prenatal_opioid_hiv","all")

### Save Output for future use

In [0]:
hcv_hiv_lab_quantative_prenatal_opioid_hiv.write.mode("overwrite").saveAsTable(f"covariate_output.hcv_hiv_lab_quantative_prenatal_opioid_hiv")
hcv_hiv_lab_quantative_prenatal_opioid_hcv.write.mode("overwrite").saveAsTable(f"covariate_output.hcv_hiv_lab_quantative_prenatal_opioid_hcv")

#### Review only:
- COMBINE HCV positive and HCV quantative (consider HCV is also positive when it got quantative value)
- COMBINE HIV positive and HIV quantative (consider HIV is also positive when it got quantative value) 

In [0]:
# def report_combination(lab_name):
    
#    sql="""
#        select *, case when (mom_person_id,baby_person_id) in (select mom_person_id,baby_person_id from hcv_hiv_lab_positive_prenatal_opioid_{lab_name}) then 1 else 0 end as already_in_hcvpositive_table
#     from hcv_hiv_lab_quantative_prenatal_opioid_{lab_name
    

#    df = spark.sql(sql)
#    return df

##### HCV report combination

In [0]:
# hcv_hiv_lab_quantative_prenatal_opioid_hcv_ifinpositive = report_combination('hcv')
# hcv_hiv_lab_quantative_prenatal_opioid_hcv_ifinpositive.createOrReplaceTempView("hcv_hiv_lab_quantative_prenatal_opioid_hcv_ifinpositive")  
# hcv_hiv_lab_quantative_prenatal_opioid_hcv_ifinpositive.display()


##### HIV report combination

In [0]:
# hcv_hiv_lab_quantative_prenatal_opioid_hiv_ifinpositive = report_combination('hiv')
# hcv_hiv_lab_quantative_prenatal_opioid_hiv_ifinpositive.createOrReplaceTempView("hcv_hiv_lab_quantative_prenatal_opioid_hiv_ifinpositive")  
# hcv_hiv_lab_quantative_prenatal_opioid_hiv_ifinpositive.display()